In [ ]:
import os
import numpy as np
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
import json
import matplotlib.pyplot as plt
from glob import glob
from transformers import BartTokenizer
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import load_metric
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import sys
sys.path.append('C:/Users/Anurag/Desktop/EEG2text/src')
from dataclasses import dataclass
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import torch
import numpy as np
from torch import optim
import logging
from datetime import datetime
from config import ModelConfig_wordlevel
from models import *
from transformers import BartTokenizer
import pickle
from common.utils.data import *
import warnings


In [11]:
class ZuCo_dataset(Dataset):
    def __init__(self, input_dataset_dicts, phase, tokenizer, subject = 'ALL', eeg_type = 'GD', bands = ['_t1','_t2','_a1','_a2','_b1','_b2','_g1','_g2'], setting = 'unique_sent', is_add_CLS_token = False, max_len : int= None):
        self.inputs = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        if not isinstance(input_dataset_dicts,list):
            input_dataset_dicts = [input_dataset_dicts]
        print(f'[INFO]loading {len(input_dataset_dicts)} task datasets')
        
        for input_dataset_dict in input_dataset_dicts:
            if subject == 'ALL':
                subjects = list(input_dataset_dict.keys())
                print('[INFO]using subjects: ', subjects)
            else:
                subjects = [subject]
            
            total_num_sentence = len(input_dataset_dict[subjects[0]])
            
            train_divider = int(0.8*total_num_sentence)
            dev_divider = train_divider + int(0.1*total_num_sentence)
            
            print(f'train divider = {train_divider}')
            print(f'dev divider = {dev_divider}')

            if setting == 'unique_sent':
                # take first 80% as trainset, 10% as dev and 10% as test
                if phase == 'train':
                    print('[INFO]initializing a train set...')
                    for key in subjects:
                        for i in range(train_divider):
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
                elif phase == 'dev':
                    print('[INFO]initializing a dev set...')
                    for key in subjects:
                        for i in range(train_divider,dev_divider):
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
                elif phase == 'test':
                    print('[INFO]initializing a test set...')
                    for key in subjects:
                        for i in range(dev_divider,total_num_sentence):
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
            elif setting == 'unique_subj':
                print('WARNING!!! only implemented for SR v1 dataset ')
                # subject ['ZAB', 'ZDM', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZKW'] for train
                # subject ['ZMG'] for dev
                # subject ['ZPH'] for test
                if phase == 'train':
                    print(f'[INFO]initializing a train set using {setting} setting...')
                    for i in range(total_num_sentence):
                        for key in ['ZAB', 'ZDM', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH','ZKW']:
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
                if phase == 'dev':
                    print(f'[INFO]initializing a dev set using {setting} setting...')
                    for i in range(total_num_sentence):
                        for key in ['ZMG']:
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
                if phase == 'test':
                    print(f'[INFO]initializing a test set using {setting} setting...')
                    for i in range(total_num_sentence):
                        for key in ['ZPH']:
                            input_sample = get_input_sample(input_dataset_dict[key][i],self.tokenizer,eeg_type,bands = bands, add_CLS_token = is_add_CLS_token)
                            if input_sample is not None:
                                self.inputs.append(input_sample)
            # print('++ adding task to dataset, now we have:', len(self.inputs))

        # print('[INFO]input tensor size:', self.inputs[0]['input_embeddings'].size())
        print()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_sample = self.inputs[idx]
        
        raw_eeg = input_sample['raw_eeg']
        if self.max_len is not None:
            # Truncate or pad raw_eeg array
            if raw_eeg.shape[1] > self.max_len:
                raw_eeg = raw_eeg[:, :self.max_len]
            elif raw_eeg.shape[1] < self.max_len:
                pad_width = ((0, 0), (0, self.max_len - raw_eeg.shape[1]))
                raw_eeg = np.pad(raw_eeg, pad_width, mode='constant', constant_values=0)
        
        # return (
        #     raw_eeg,
        #     input_sample['target_string'],
        #     input_sample['target_ids'], 
        #     input_sample['target_mask'],  
        # )
        return (
            raw_eeg,
            input_sample['target_string'],
            input_sample['target_ids'], 
            input_sample['target_mask'],  
            input_sample['input_embeddings'], # new
            input_sample['seq_len'], # new
            input_sample['input_attn_mask'], 
            input_sample['input_attn_mask_invert'],
        )

In [13]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
dataset_setting = 'unique_sent'
subject_choice = 'ALL'
eeg_type_choice = 'GD'
bands_choice = ['_t1','_t2','_a1','_a2','_b1','_b2','_g1','_g2'] 

config=ModelConfig_wordlevel()

In [4]:
print('Creating whole_dataset_dicts...')
whole_dataset_dicts = []
dataset_path_task1 = '/nlsasfs/home/nltm-st/sujitk/temp/eeg2text/datasets/pickle/task1-SR-1.0/task1-SR-1.0-dataset.pickle' 
with open(dataset_path_task1, 'rb') as handle:
    whole_dataset_dicts.append(pickle.load(handle))
print('whole_dataset_dicts created')

Creating whole_dataset_dicts...
whole_dataset_dicts created


In [5]:
def prepare_dataloader(dataset: Dataset):
    return DataLoader(
        dataset,
        drop_last=False,
        batch_size=1,
        shuffle=False,
        # sampler=DistributedSampler(dataset, shuffle=True)
    )


In [8]:
# for t in ['train','dev','test']:
#     dset = ZuCo_dataset(whole_dataset_dicts, t, tokenizer, subject = subject_choice, eeg_type = eeg_type_choice, bands = bands_choice, setting = dataset_setting, max_len=config.time_len)
#     dl = prepare_dataloader(dset)
#     batch_iterator = tqdm(dl)

In [6]:
dset = ZuCo_dataset(whole_dataset_dicts, 'train', tokenizer, subject = subject_choice, eeg_type = eeg_type_choice, bands = bands_choice, setting = dataset_setting, max_len=config.time_len)

[INFO]loading 1 task datasets
[INFO]using subjects:  ['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZKW', 'ZMG', 'ZPH']
train divider = 320
dev divider = 360
[INFO]initializing a train set...



In [7]:
dl = prepare_dataloader(dset)
batch_iterator = tqdm(dl)

  0%|          | 0/3609 [00:00<?, ?it/s]

In [25]:
i=1
for raw_eeg, target_string, target_ids, target_mask, word_level_embed, orig_seq_len, input_attn_mask, input_attn_mask_invert  in batch_iterator:
    raw_eeg = raw_eeg.to(torch.float32) #.to(self.gpu_id)
    # print(f'raw_eeg.shape: {raw_eeg.shape}, raw_eeg: {raw_eeg}')
    # print(torch.isnan(raw_eeg).any(), end='\n\n')
    tr=torch.isnan(raw_eeg)
    print(torch.nonzero(torch.sum(tr,dim=2))==torch.tensor([[  0, 104]]))
    
    i+=1
    if(i>=50):
        break

tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[True, True]])
tensor([[Tr

In [27]:
total_count=len(dset)
last_channel_nan_count=0
for raw_eeg, target_string, target_ids, target_mask, word_level_embed, orig_seq_len, input_attn_mask, input_attn_mask_invert  in batch_iterator:
    raw_eeg = raw_eeg.to(torch.float32) #.to(self.gpu_id)
    # print(f'raw_eeg.shape: {raw_eeg.shape}, raw_eeg: {raw_eeg}')
    # print(torch.isnan(raw_eeg).any(), end='\n\n')
    tr=torch.isnan(raw_eeg)
    last_channel_nan=(torch.nonzero(torch.sum(tr,dim=2))==torch.tensor([[  0, 104]]))
    last_channel_nan_count+=last_channel_nan
    i+=1

print(f'total_count: {total_count}, last_channel_nan_count: {last_channel_nan_count}')

total_count: 3609, last_channel_nan_count: tensor([[3609, 3609]])


In [28]:
last_channel_nan

tensor([[True, True]])

In [12]:
tr

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False]]])

In [21]:
trs=torch.sum(tr,dim=2)
trs

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0, 3341]])

In [23]:
torch.nonzero(trs)

tensor([[  0, 104]])

In [26]:
len(dset)

3609

In [16]:
i=1
for raw_eeg, target_string, target_ids, target_mask, word_level_embed, orig_seq_len, input_attn_mask, input_attn_mask_invert  in batch_iterator:
    raw_eeg = raw_eeg.to(torch.float32) #.to(self.gpu_id)
    # print(f'raw_eeg.shape: {raw_eeg.shape}, raw_eeg: {raw_eeg}')
    print(torch.isnan(raw_eeg).any(), end=', ')
    if(i%10==0): print()
    i+=1
    if(i>=50):
        break

tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 
tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 
tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 
tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 
tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), tensor(True), 

In [19]:
print('Creating whole_dataset_dicts...')
zuco2 = []
dataset_path_task2_v2 = '/nlsasfs/home/nltm-st/sujitk/temp/eeg2text/datasets/pickle/task2-NR-2.0/task2-NR-2.0-dataset.pickle' 
with open(dataset_path_task2_v2, 'rb') as handle:
    zuco2.append(pickle.load(handle))
print('whole_dataset_dicts created')

Creating whole_dataset_dicts...
whole_dataset_dicts created


In [21]:
dset2 = ZuCo_dataset(zuco2, 'dev', tokenizer, subject = subject_choice, eeg_type = eeg_type_choice, bands = bands_choice, setting = dataset_setting, max_len=config.time_len)

[INFO]loading 1 task datasets
[INFO]using subjects:  ['YSD', 'YDR', 'YRK', 'YFS', 'YAG', 'YIS', 'YLS', 'YRH', 'YHS', 'YMS', 'YSL', 'YDG', 'YFR', 'YRP', 'YTL', 'YAC', 'YAK', 'YMD']
train divider = 279
dev divider = 313
[INFO]initializing a dev set...



In [22]:
dl2 = prepare_dataloader(dset2)
batch_iterator2 = tqdm(dl2)

  0%|          | 0/522 [00:00<?, ?it/s]

In [23]:
i=1
for raw_eeg, target_string, target_ids, target_mask, word_level_embed, orig_seq_len, input_attn_mask, input_attn_mask_invert  in batch_iterator2:
    raw_eeg = raw_eeg.to(torch.float32) #.to(self.gpu_id)
    # print(f'raw_eeg.shape: {raw_eeg.shape}, raw_eeg: {raw_eeg}')
    print(torch.isnan(raw_eeg).any(), end=', ')
    if(i%10==0): print()
    i+=1
    if(i>=50):
        break

  0%|          | 1/522 [00:04<37:01,  4.26s/it]

tensor(False), 

  2%|▏         | 11/522 [00:04<02:02,  4.18it/s]

tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), 
tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), 

  3%|▎         | 18/522 [00:04<01:04,  7.80it/s]

...

tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), 


  8%|▊         | 40/522 [00:05<00:27, 17.78it/s]

tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), 


  9%|▉         | 48/522 [00:06<01:00,  7.89it/s]


tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), tensor(False), 

In [14]:
objects = []
with (open("NR-1.0", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [23]:
# Different subjects
objects[0].keys()

dict_keys(['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZKW', 'ZMG', 'ZPH'])

In [25]:
objects[0]['ZAB'][0].keys()

dict_keys(['content', 'sentence_level_EEG', 'raw_eeg', 'word', 'word_tokens_has_fixation', 'word_tokens_with_mask', 'word_tokens_all'])

In [ ]:
objects[0]['ZAB'][0]['sentence_level_EEG'] # each of 8 bands (frequency)

In [29]:
len(objects[0]['ZAB'][0]['sentence_level_EEG']['mean_a1'])

105